Импортирую библиотеки

In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd

Пишу ИНН нужной компании поставщика

In [ ]:
LAW = 'fz44'

# в поле INN нужно ввести 10-значный номер ИНН

INN = ##############
NUMBER_OF_RESULTS = '100'

Выбираю закон

In [ ]:
# 44-ФЗ

All_contracts = 'https://zakupki.gov.ru/epz/contract/search/results.html?morphology=on\
&'+ LAW + '=on&contractStageList_0=on&contractStageList_1=on&contractStageList_2=on&contractStageList_3=on\
&contractStageList=0%2C1%2C2%2C3&contractPriceFrom=0&contractPriceTo=200000000000&contractCurrencyID=-1\
&budgetaryFunds=on&extraBudgetaryFunds=on&customerPlaceWithNested=on&selectedContractDataChanges=ANY\
&supplierTitle=' + INN + '&countryRegIdsWithNested=on&classifiersMpGroupId=0&sortBy=PO_DATE_OBNOVLENIJA\
&pageNumber=1&sortDirection=false&recordsPerPage=_' + NUMBER_OF_RESULTS

# 223-ФЗ

# All_contracts = 'https://zakupki.gov.ru/epz/contractfz223/search/results.html?morphology=on\
# &search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&hiddenFz223=true\
# &supplierTitle=' + INN +'&sortBy=BY_UPDATE_DATE&pageNumber=1&sortDirection=false&recordsPerPage=_' + NUMBER_OF_RESULTS

Запрашиваю данные из ссылки выше

In [ ]:
page = requests.get(All_contracts).content
html_tree = html.fromstring(page.decode("utf-8", "replace"))
soup = BeautifulSoup(page, 'html.parser')

Создаю список из номеров контрактов выбранного поставщика

In [ ]:
contracts_id = []
divs = soup.find_all('div', attrs={'class': 'registry-entry__header-mid__number'})
for div in divs:
    contracts_id.append(div.text.strip().replace('№ ', ''))


По каждому номеру из списка выше собираю все возможные данные с zakupki.gov.ru

In [ ]:
all_data = {}
for number, contract_id in enumerate(contracts_id):
    page = requests.get('https://zakupki.gov.ru/epz/contract/contractCard/common-info.html?reestrNumber=' + contract_id).content
    html_tree = html.fromstring(page.decode("utf-8", "replace"))

    items_list = html_tree.xpath("//div[@class='blockInfo__section']")

    soup = BeautifulSoup(page, 'html.parser')

    divs = soup.find_all('section', attrs={'class': 'blockInfo__section section'})

    data = {}
    div_length = len(divs)
    for i in range(div_length):
        text = divs[i].find('span', attrs={'class': 'section__title'}).text.replace('\xa0', ' ').replace('\r\n', '').replace('  ', ' ')
        value = divs[i].find('span', attrs={'class': 'section__info'}).text.strip().replace('\xa0', ' ').replace('\r\n', '').replace('  ', ' ')
        data.update({text:value})
    all_data.update({number + 1:data})

Перевожу собранные данные в DataFrame

In [ ]:
df = pd.DataFrame(all_data).T

Смотрю что получилось

In [ ]:
df

Выбираю нужные заголовки

In [ ]:
df_current = df[['Сокращенное наименование заказчика', 
    'Номер контракта', 
    'Дата заключения контракта',
    'Дата начала исполнения контракта',
    'Дата окончания исполнения контракта',
    'Предмет контракта',
    'Цена контракта'
   ]]

Смотрю на итоговый результат

In [ ]:
df_current

Сохраняю в Excel

In [ ]:
df_current.to_excel("output.xlsx")